In [1]:
'''
:@Author: Remi
:@Date: 2023/9/13 08:21:53
:@LastEditors: Remi
:@LastEditTime: 2023/12/17 13:08:51
:Description: 
'''
import os
import numpy as np
import preprocessing as pp
import file_tool as ft

path = r'D:\20231218'
path_list = ft.find_file(path,'.csv')

filename = [os.path.basename(path_list[i])[:-4] for i in range(len(path_list))]####保留文件名


for path_num,paths in enumerate(path_list):
    wl,spec = ft.read_ava(paths)
    rsd = []

    spec = pp.Anomalous_spectrum_removal(spec[:,:])
    wl = wl[:]#################################################################切片了，不切的删掉这行
    spec_num = len(spec[0,:])
    
    rsd.append(pp.RSD_calculate(spec,spec_num))
    #导入对应一个波长的所有光谱数据，平均将光谱分为n份的数量


    ##########weighted method
    ##############genrate slope map#########
    ###################feather devide###########    
    for i in range(spec_num):#calculate slope
        
        spec[:,i] = pp.Normalization(spec[:,i],method='total_intensity')
        
        single_spec = spec[:,i]
        slope = np.int64((np.append(single_spec,single_spec[-1]) - np.insert(single_spec,0,single_spec[0]))>= 0)#eg  0123 vs 0 -1 1 1 0 ,-1 is the slope between num0&1
        peak = np.insert(slope,0,slope[0]) - np.append(slope,slope[-1])
        if i == 0:
            slope_map = slope
            peak_map = peak
        else:
            slope_map = np.vstack((slope_map,slope)) 
            peak_map = np.vstack((peak_map,peak)) 
            #map 1 = + while 0 = -
    
    slope_map[slope_map == 0] = - 1
    peak_map = peak_map[:,1:-1].T###计算peak形状

    for single_ in slope_map:###计算 arise 和 decay 形状
        for i in range(len(single_)-1):
            if single_[i] != single_[i+1]:
                single_[i] = 0

    scores =  (np.abs(np.sum(peak_map,axis = 1)) +  np.abs(np.sum(slope_map[:,:-1],axis = 0)))/spec_num
    mean_spec = np.mean(spec,axis = 1)

###########other process method
    '''
    different weighed method
    '''
    #weighter_spec = mean_spec * np.log((scores * (math.e - 1) + 1))
    weighter_spec = mean_spec * scores

    #slice_wl,slice_spec = pp.spec_slice([50,50],[50,50],[11544,11644],wl,spec)
    #slice_wl = slice_wl.flatten()


#########for save
    if path_num == 0:
        wl_list = np.hstack((np.array(([['wl']])).T,wl.T))
        mean_spec_list = np.hstack((filename[path_num],mean_spec))
    else:
        mean_spec_list = np.vstack((mean_spec_list,np.hstack((filename[path_num],mean_spec))))

#np.savetxt(path + '//' +filename[path_num]+ 'sliced.csv',np.hstack((np.array(([slice_wl])).T,slice_spec)),delimiter=',',fmt='%.04f')
#np.savetxt(path + '//' +filename[path_num]+ 'wei.csv',np.hstack((wl,np.vstack((mean_spec,weighter_spec)).T)),delimiter=',',fmt='%.04f')
np.savetxt(path + '//' + 'normalized.csv',np.vstack((wl_list,mean_spec_list)).T,delimiter=',',fmt = '%s')

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

np.set_printoptions(precision = 6)

cmap=mpl.cm.RdYlBu_r#获取色条
newcolors=cmap(np.linspace(0,1,256))#分片操作
newcmap=ListedColormap(newcolors[100:])#只取100之后的颜色列表，前面的舍去

peak_list = [779.898,852.075]##############peak position
tol = 1#算峰面积的容差
conc = []

##################path##########################
filename = r'D:\20231218\normalized.csv'#file position
savepath = r'D:\20221218'
##################load#########################

wl,spec=ft.Load_MultiSpec(filename)
wl = wl[1:]
spec = spec[1:,:]
#meanspec = np.mean(spec,axis = 1)############while variable no_grad use mean spec , otherwise chose max spec 
maxspec = np.max(spec,axis = 1)

p,m,peak_list = pp.Peak_integrate(wl,maxspec,peak_list,tol)

##################calculate#####################
for i in range(len(spec[0])):
    for j in range(len(peak_list)):#pm wavelength

        slice_cache = spec[peak_list[j] - m[j]:peak_list[j] + p[j],i]
        slice_square = (p[j] + m[j] + 1) * (slice_cache[0] + slice_cache[-1])/2
        if j == 0:
            slice_ = slice_cache.sum(axis=0) - slice_square
        else:
            slice_ = np.vstack((slice_,(slice_cache.sum(axis=0) - slice_square)))

        if i == 0:
            conc.append(wl[peak_list[j]])

    if i == 0:
        slice_sample = slice_   
    else: 
        slice_sample = np.hstack((slice_sample,slice_))

###################save#####################################

#np.savetxt(savepath +'\\' + 'square.csv',np.hstack((np.array([conc]).T,slice_sample)).T,delimiter=',',fmt='%.04f')  

###################plot#####################################

for j in range(len(peak_list)):
    sns_spec = spec[:][peak_list[j] - m[j]:peak_list[j] + p[j],:]
    sns_wl = wl_cache = wl[peak_list[j] - m[j]:peak_list[j] + p[j]]

##################减阴影
    for i in range(len(sns_spec[0,:])):
        slope = (sns_spec[:,i][0] - sns_spec[:,i][-1])/(sns_wl[0] - sns_wl[-1])
        bias = sns_spec[:,i][0] - slope * sns_wl[0]
        sns_spec[:,i] = sns_spec[:,i] - (sns_wl * slope + bias)

##################展区间
    for i in range(len(sns_spec[0][:]) - 1):
        sns_wl = np.column_stack((sns_wl,wl_cache))

    sns_spec = sns_spec.flatten()
    sns_wl = sns_wl.flatten()
    data = np.column_stack((sns_wl, sns_spec))
    df = pd.DataFrame(data, columns=['wl', 'int'])
    sns.relplot(x="wl", y="int", kind="line", data=df,ci = "sd")
    sns.relplot(x="wl", y="int", kind="line", data=df)
    plt.show()

NameError: name 'np' is not defined